### Is it possible to build a classification model for detecting prime numbers?

In [1]:
# imports
import numpy as np
#import matplotlib as plt
import pandas as pd

In [2]:
# params
prime_lim = 500000

In [3]:
# read in prime numbers
primes = np.load(f'../../artifacts/primes/prime_{prime_lim}.npy')
primes[:100]

array([  2,   3,   5,   7,  11,  13,  17,  19,  23,  29,  31,  37,  41,
        43,  47,  53,  59,  61,  67,  71,  73,  79,  83,  89,  97, 101,
       103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167,
       173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239,
       241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,
       317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397,
       401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467,
       479, 487, 491, 499, 503, 509, 521, 523, 541], dtype=int64)

In [4]:
# convert to natural numbers with binary target
natural_numbers = np.arange(0,prime_lim)
target = np.zeros(prime_lim, dtype=bool)
target[primes] = True

In [5]:
data = pd.DataFrame(data={'n': natural_numbers[2:], 'y': target[2:]})

### Features

what kind of features could we have?
- n+1, n-1, ... we can extend this a lot
- 2n
- n*2
- n/2

right now, idea would be to have every row independent from each other 
-> model is not supposed to actually compute the prime numbers (although it would be interesting to know whether it could)

In [6]:
data['n+1'] = data['n'].apply(lambda x: x+1)
data['n-1'] = data['n'].apply(lambda x: x-1)
data['2n'] = data['n'].apply(lambda x: x*2)
data['n**2'] = data['n'].apply(lambda x: x**2)
data['n%2'] = data['n'].apply(lambda x: x%2) # this might be too strong as an indicator?

In [7]:
# distance to last prime?
# number of primes before this number
# dividing current number by last prime? 
# what is last prime?

data['last_prime']=data['n'].apply(lambda x: primes[primes<x].max() if x!=2 else -1)
data['primes_lower_n']=data['n'].apply(lambda x: len(primes[primes<x]) if x!=2 else 0)
data['n_div_last_prime']=data.apply(lambda x: x['n']/x['last_prime'] if x['n']!=2 else -1, axis=1)
data['n_minus_last_prime']=data.apply(lambda x: x['n']-x['last_prime'] if x['n']!=2 else -1, axis=1)


# I could also try out different mod, like n/int((n/3)), is this different from n/3?

In [8]:
data.head(15)

,n,y,n+1,n-1,2n,n**2,n%2,last_prime,primes_lower_n,n_div_last_prime,n_minus_last_prime
0,2,True,3,1,4,4,0,-1,0,-1.000000,-1
1,3,True,4,2,6,9,1,2,1,1.500000,1
2,4,False,5,3,8,16,0,3,2,1.333333,1
3,5,True,6,4,10,25,1,3,2,1.666667,2
4,6,False,7,5,12,36,0,5,3,1.200000,1
5,7,True,8,6,14,49,1,5,3,1.400000,2
6,8,False,9,7,16,64,0,7,4,1.142857,1
7,9,False,10,8,18,81,1,7,4,1.285714,2
8,10,False,11,9,20,100,0,7,4,1.428571,3
9,11,True,12,10,22,121,1,7,4,1.571429,4


### First Model: Logistic Regression

In [9]:
feature_col = data.columns
target_col = 'y'
feature_col = feature_col.drop(target_col)


In [10]:
# thats obviously crucial if you want the model to converge...

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[feature_col] = scaler.fit_transform(data[feature_col])
           

In [11]:
X, y = data[feature_col], data[target_col]


In [12]:
# divide in train and test randomly

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
# train logistic regression as start

from sklearn.linear_model import LogisticRegressionCV

# lbfgs solver, l2 penalty
clf = LogisticRegressionCV(cv=10, random_state=0, max_iter=500).fit(X_train, y_train)

In [14]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm # no primes predicted at all

array([[151208,      0],
       [ 13792,      0]], dtype=int64)

---> features are probably not strong enough to help with the identification of primes